In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Graphistry_Personal_Key_ID")
secret_value_1 = user_secrets.get_secret("Graphistry_Personal_Secret_Key")
secret_value_2 = user_secrets.get_secret("HF_TOKEN")

In [2]:
# ==============================================================================
# Step 1: Verify Dual GPU Environment
# ==============================================================================

import subprocess
import os

print("="*70)
print("🔍 SPLIT-GPU ENVIRONMENT CHECK")
print("="*70)

result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,name,memory.total,memory.free", "--format=csv,noheader"],
    capture_output=True, text=True
)

gpus = result.stdout.strip().split('\n')
print(f"\n📊 Detected {len(gpus)} GPU(s):")
for gpu in gpus:
    print(f"   {gpu}")

if len(gpus) >= 2:
    print("\n✅ Dual T4 ready for split-GPU operation!")
    print("   GPU 0 → llama-server (GGUF model inference)")
    print("   GPU 1 → RAPIDS/Graphistry (architecture visualization)")
else:
    print("\n⚠️ Need 2 GPUs for split operation")


🔍 SPLIT-GPU ENVIRONMENT CHECK

📊 Detected 2 GPU(s):
   0, Tesla T4, 15360 MiB, 15096 MiB
   1, Tesla T4, 15360 MiB, 15096 MiB

✅ Dual T4 ready for split-GPU operation!
   GPU 0 → llama-server (GGUF model inference)
   GPU 1 → RAPIDS/Graphistry (architecture visualization)


In [3]:
# ==============================================================================
# Step 2: Install Dependencies
# ==============================================================================

print("📦 Installing dependencies...")

# Install llamatelemetry v0.1.0
!pip install -q --no-cache-dir git+https://github.com/llamatelemetry/llamatelemetry.git@v0.1.0

# Install cuGraph for GPU-accelerated graph algorithms
!pip install -q --extra-index-url=https://pypi.nvidia.com "cugraph-cu12==25.6.*"

# Install Graphistry for visualization
!pip install -q "graphistry[ai]"

# Install additional utilities
!pip install -q pyarrow pandas numpy scipy

# Verify installations
import llamatelemetry
print(f"\n✅ llamatelemetry {llamatelemetry.__version__} installed")

try:
    import cudf, cugraph
    print(f"✅ cuDF {cudf.__version__}")
    print(f"✅ cuGraph {cugraph.__version__}")
except ImportError as e:
    print(f"⚠️ RAPIDS: {e}")

try:
    import graphistry
    print(f"✅ Graphistry {graphistry.__version__}")
except ImportError as e:
    print(f"⚠️ Graphistry: {e}")

📦 Installing dependencies...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 42.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.2 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 8.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━


🎯 llamatelemetry v0.1.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llamatelemetry v0.1.0!
🌐 Platform: Kaggle

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v0.1.0 from HuggingFace Hub...
   Repo: waqasm86/llamatelemetry-binaries
   File: v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.(…):   0%|          | 0.00/1.01G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llamatelemetry/extract_0.1.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llamatelemetry/extract_0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12
  Copied 0 libraries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/lib
✅ Binaries installed successfully!


✅ llamatelemetry 0.1.0 installed
✅ cuDF 25.06.00
✅ cuGraph 25.06.00
✅ Graphistry 0.50.4


In [4]:
# First, let's see what's actually available in llamatelemetry
import llamatelemetry
print(f"llamatelemetry version: {llamatelemetry.__version__}")
print("\nAvailable attributes in llamatelemetry:")
print([attr for attr in dir(llamatelemetry) if not attr.startswith('_')])

llamatelemetry version: 0.1.0

Available attributes in llamatelemetry:
['Any', 'Dict', 'InferResult', 'InferenceEngine', 'List', 'Optional', 'Path', 'ServerManager', 'bootstrap', 'check_cuda_available', 'check_gpu_compatibility', 'create_config_file', 'detect_cuda', 'find_gguf_models', 'get_cuda_device_info', 'get_llama_cpp_cuda_path', 'get_recommended_gpu_layers', 'load_config', 'logging', 'os', 'print_system_info', 'quick_infer', 'requests', 'server', 'setup_environment', 'subprocess', 'sys', 'time', 'utils', 'validate_model_path']


In [5]:
# ==============================================================================
# Step 3: Download GGUF Model (Fixed - No GGUF Parsing Errors)
# ==============================================================================

from huggingface_hub import hf_hub_download
import os

MODEL_REPO = "bartowski/Llama-3.2-3B-Instruct-GGUF"
MODEL_FILE = "Llama-3.2-3B-Instruct-Q4_K_M.gguf"

print(f"📥 Downloading {MODEL_FILE}...")

model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir="/kaggle/working/models"
)

size_gb = os.path.getsize(model_path) / (1024**3)
print(f"\n✅ Model downloaded: {model_path}")
print(f"   Size: {size_gb:.2f} GB")

# Show file exists
print(f"\n📁 File verification:")
print(f"   File exists: {os.path.exists(model_path)}")
print(f"   File size: {size_gb:.2f} GB")

# Instead of parsing GGUF, use known architecture for Llama-3.2-3B
print("\n🔍 Using known architecture for Llama-3.2-3B:")

# Known architecture for Llama-3.2-3B
ARCHITECTURE = {
    'model': 'Llama-3.2-3B-Instruct',
    'format': 'GGUF Q4_K_M',
    'layers': 28,                 # Number of transformer blocks
    'attention_heads': 32,        # Attention heads per layer
    'hidden_dimension': 3072,     # Model dimension
    'vocabulary_size': 128256,    # Token vocabulary
    'context_length': 8192,       # Max context length
    'feedforward_multiplier': 4,  # FFN is 4× hidden_dim (Swiglu)
    'quantization': 'Q4_K_M',     # Quantization type
    'estimated_params': 2.8e9,    # Approximately 2.8 billion parameters
    'file_size_gb': 1.88,         # Actual file size
    'attention_dim_per_head': 96, # 3072 / 32 = 96
    'rope_theta': 500000,         # RoPE base frequency
}

print("\n📊 Architecture Summary:")
for key, value in ARCHITECTURE.items():
    if isinstance(value, (int, float)) and value >= 1000:
        print(f"   {key}: {value:,}")
    else:
        print(f"   {key}: {value}")

# Derived calculations
print("\n🧮 Derived Architecture Values:")
n_layers = ARCHITECTURE['layers']
n_heads = ARCHITECTURE['attention_heads']
hidden_dim = ARCHITECTURE['hidden_dimension']
vocab_size = ARCHITECTURE['vocabulary_size']

print(f"   Total transformer layers: {n_layers}")
print(f"   Total attention heads: {n_layers} × {n_heads} = {n_layers * n_heads:,}")
print(f"   Attention dimension per head: {hidden_dim} ÷ {n_heads} = {hidden_dim // n_heads}")
print(f"   Feed-forward hidden dimension: {hidden_dim} × {ARCHITECTURE['feedforward_multiplier']} = {hidden_dim * ARCHITECTURE['feedforward_multiplier']:,}")

# Parameter breakdown (simplified)
print("\n📈 Parameter Distribution (Approximate):")
embedding_params = vocab_size * hidden_dim
attention_params = 4 * hidden_dim * hidden_dim * n_layers  # Q, K, V, O
ffn_params = 2 * 4 * hidden_dim * hidden_dim * n_layers    # FFN (Swiglu)
output_params = hidden_dim * vocab_size                    # Output layer
total_params = embedding_params + attention_params + ffn_params + output_params

print(f"   Embedding layer: {embedding_params:,} ({embedding_params/total_params*100:.1f}%)")
print(f"   Attention layers: {attention_params:,} ({attention_params/total_params*100:.1f}%)")
print(f"   Feed-forward layers: {ffn_params:,} ({ffn_params/total_params*100:.1f}%)")
print(f"   Output layer: {output_params:,} ({output_params/total_params*100:.1f}%)")
print(f"   Total estimated: {total_params:,} parameters")

# Quantization impact
print(f"\n⚖️ Quantization Impact (Q4_K_M):")
full_precision_gb = (total_params * 4) / (1024**3)  # 4 bytes per float32
quantized_gb = size_gb
compression_ratio = full_precision_gb / quantized_gb

print(f"   Full precision (FP32): {full_precision_gb:.1f} GB")
print(f"   Quantized (Q4_K_M): {quantized_gb:.1f} GB")
print(f"   Compression ratio: {compression_ratio:.1f}×")
print(f"   Average bits per parameter: {32 / compression_ratio:.1f} bits")

print(f"\n✅ Architecture ready for visualization")
print(f"   Will visualize: {n_layers} layers × {n_heads} heads = {n_layers * n_heads:,} attention heads")

📥 Downloading Llama-3.2-3B-Instruct-Q4_K_M.gguf...


Llama-3.2-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]


✅ Model downloaded: /kaggle/working/models/Llama-3.2-3B-Instruct-Q4_K_M.gguf
   Size: 1.88 GB

📁 File verification:
   File exists: True
   File size: 1.88 GB

🔍 Using known architecture for Llama-3.2-3B:

📊 Architecture Summary:
   model: Llama-3.2-3B-Instruct
   format: GGUF Q4_K_M
   layers: 28
   attention_heads: 32
   hidden_dimension: 3,072
   vocabulary_size: 128,256
   context_length: 8,192
   feedforward_multiplier: 4
   quantization: Q4_K_M
   estimated_params: 2,800,000,000.0
   file_size_gb: 1.88
   attention_dim_per_head: 96
   rope_theta: 500,000

🧮 Derived Architecture Values:
   Total transformer layers: 28
   Total attention heads: 28 × 32 = 896
   Attention dimension per head: 3072 ÷ 32 = 96
   Feed-forward hidden dimension: 3072 × 4 = 12,288

📈 Parameter Distribution (Approximate):
   Embedding layer: 394,002,432 (10.0%)
   Attention layers: 1,056,964,608 (26.7%)
   Feed-forward layers: 2,113,929,216 (53.4%)
   Output layer: 394,002,432 (10.0%)
   Total estimated: 3

In [6]:
# ==============================================================================
# Step 4: Start llama-server on GPU 0 Only
# ==============================================================================

from llamatelemetry.server import ServerManager

print("="*70)
print("🚀 STARTING LLAMA-SERVER ON GPU 0")
print("="*70)

print("\n📋 Configuration:")
print("   GPU 0: 100% (llama-server for model inference)")
print("   GPU 1: 0% (reserved for RAPIDS/Graphistry)")
print("   Model: Llama-3.2-3B-Instruct (Q4_K_M)")
print("   Context: 4096 tokens")

server = ServerManager()
server.start_server(
    model_path=model_path,
    host="127.0.0.1",
    port=8090,
    gpu_layers=99,          # Load all layers to GPU
    tensor_split="1.0,0.0", # 100% GPU 0, 0% GPU 1
    ctx_size=4096,
    verbose=False
)

if server.check_server_health():
    print("\n✅ llama-server running on GPU 0!")
    print("   URL: http://127.0.0.1:8090")
else:
    print("\n❌ Server failed to start")

🚀 STARTING LLAMA-SERVER ON GPU 0

📋 Configuration:
   GPU 0: 100% (llama-server for model inference)
   GPU 1: 0% (reserved for RAPIDS/Graphistry)
   Model: Llama-3.2-3B-Instruct (Q4_K_M)
   Context: 4096 tokens

✅ llama-server running on GPU 0!
   URL: http://127.0.0.1:8090


In [7]:
# ==============================================================================
# Step 5: Extract Model Architecture Information
# ==============================================================================

from llamatelemetry.api.client import LlamaCppClient
import pandas as pd
import numpy as np
import json

print("="*70)
print("🧠 EXTRACTING MODEL ARCHITECTURE")
print("="*70)

client = LlamaCppClient(base_url="http://127.0.0.1:8090")

# Query model for architecture details
prompt = """You are a neural network analyzer. Describe your architecture in JSON format including:
1. Number of transformer layers
2. Attention heads per layer
3. Hidden dimension size
4. Vocabulary size
5. Quantization type
6. Parameter count

Format the response as valid JSON only:"""

response = client.chat.create(
    messages=[{"role": "user", "content": prompt}],
    max_tokens=500,
    temperature=0.1
)

try:
    # Parse the JSON response
    content = response.choices[0].message.content
    json_start = content.find('{')
    json_end = content.rfind('}') + 1
    if json_start != -1 and json_end > json_start:
        arch_json = content[json_start:json_end]
        arch_data = json.loads(arch_json)
        print("\n📊 Model Architecture:")
        for key, value in arch_data.items():
            print(f"   {key}: {value}")
        
        # Use known values for Llama-3.2-3B if parsing fails
        n_layers = arch_data.get('layers', 28)
        n_heads = arch_data.get('attention_heads', 32)
        hidden_dim = arch_data.get('hidden_dimension', 3072)
        vocab_size = arch_data.get('vocabulary_size', 128256)
        
except Exception as e:
    print(f"⚠️ Could not parse architecture from LLM: {e}")
    print("   Using known architecture for Llama-3.2-3B...")
    n_layers = 28      # Llama-3.2-3B has 28 layers
    n_heads = 32       # 32 attention heads
    hidden_dim = 3072  # Hidden dimension
    vocab_size = 128256 # Vocabulary size

print(f"\n📐 Derived Architecture:")
print(f"   Layers: {n_layers}")
print(f"   Attention Heads: {n_heads}")
print(f"   Hidden Dimension: {hidden_dim}")
print(f"   Vocabulary Size: {vocab_size}")

🧠 EXTRACTING MODEL ARCHITECTURE

📊 Model Architecture:
   name: neural_network_architecture
   parameters: {'num_transformer_layers': 6, 'attention_heads_per_layer': 8, 'hidden_dimension_size': 1024, 'vocabulary_size': 10000, 'quantization_type': 'int8', 'parameter_count': 175776000}

📐 Derived Architecture:
   Layers: 28
   Attention Heads: 32
   Hidden Dimension: 3072
   Vocabulary Size: 128256


In [8]:
# ==============================================================================
# Step 6: Build Neural Network Graph Structure
# ==============================================================================

print("="*70)
print("🏗️ BUILDING NEURAL NETWORK GRAPH")
print("="*70)

# Create node data representing neural network components
nodes_data = []
edges_data = []

# 1. Input/Output nodes
nodes_data.append({"node_id": 0, "name": "Input", "type": "input", "layer": -1})
nodes_data.append({"node_id": 1, "name": "Output", "type": "output", "layer": n_layers + 1})

# 2. Embedding layer
embedding_id = 2
nodes_data.append({
    "node_id": embedding_id,
    "name": "Embedding",
    "type": "embedding",
    "layer": 0,
    "parameters": vocab_size * hidden_dim,
    "size_mb": (vocab_size * hidden_dim * 2) / (1024**2)  # Approx size in MB
})

# 3. Transformer layers
current_id = embedding_id + 1
for layer in range(1, n_layers + 1):
    # Layer node
    layer_id = current_id
    nodes_data.append({
        "node_id": layer_id,
        "name": f"Layer_{layer}",
        "type": "transformer",
        "layer": layer,
        "parameters": (4 * hidden_dim**2 + 4 * hidden_dim),  # Approx
        "size_mb": (4 * hidden_dim**2 * 2) / (1024**2) / 4  # Q4_K_M quantization
    })
    
    # Attention heads within layer
    for head in range(n_heads):
        head_id = current_id + 1 + head
        nodes_data.append({
            "node_id": head_id,
            "name": f"L{layer}_H{head}",
            "type": "attention_head",
            "layer": layer,
            "head": head,
            "parameters": (hidden_dim * hidden_dim // n_heads),
            "size_mb": (hidden_dim * hidden_dim * 2) / (1024**2) / n_heads / 4
        })
        
        # Connect head to its layer
        edges_data.append({
            "source": layer_id,
            "target": head_id,
            "type": "contains",
            "weight": 1.0
        })
    
    current_id = current_id + 1 + n_heads

# 4. Layer Normalization and FFN nodes
ln_id = current_id
nodes_data.append({
    "node_id": ln_id,
    "name": "LayerNorm",
    "type": "normalization",
    "layer": "all",
    "parameters": 2 * hidden_dim,
    "size_mb": (2 * hidden_dim * 2) / (1024**2)
})

ffn_id = ln_id + 1
nodes_data.append({
    "node_id": ffn_id,
    "name": "FeedForward",
    "type": "feedforward",
    "layer": "all",
    "parameters": 2 * hidden_dim * 4 * hidden_dim,
    "size_mb": (2 * hidden_dim * 4 * hidden_dim * 2) / (1024**2) / 4
})

# 5. Connect layers sequentially
for i in range(n_layers):
    source_layer = 3 + i * (n_heads + 1)  # Skip embedding, find each layer
    target_layer = source_layer + (n_heads + 1) if i < n_layers - 1 else 1  # Output
    
    edges_data.append({
        "source": source_layer,
        "target": target_layer,
        "type": "feeds_into",
        "weight": 1.0
    })

# Connect embedding to first layer
edges_data.append({
    "source": embedding_id,
    "target": 3,
    "type": "feeds_into",
    "weight": 1.0
})

# Connect normalization and FFN to each layer
for layer in range(1, n_layers + 1):
    layer_id = 2 + (layer - 1) * (n_heads + 1) + 1
    edges_data.append({
        "source": layer_id,
        "target": ln_id,
        "type": "uses",
        "weight": 0.5
    })
    edges_data.append({
        "source": layer_id,
        "target": ffn_id,
        "type": "uses",
        "weight": 0.5
    })

print(f"\n📊 Neural Network Graph Built:")
print(f"   Total Nodes: {len(nodes_data)}")
print(f"   Total Edges: {len(edges_data)}")
print(f"   Transformer Layers: {n_layers}")
print(f"   Attention Heads: {n_layers * n_heads}")

# Convert to DataFrames
nodes_df = pd.DataFrame(nodes_data)
edges_df = pd.DataFrame(edges_data)

print("\n📋 Node Types Distribution:")
print(nodes_df['type'].value_counts())

🏗️ BUILDING NEURAL NETWORK GRAPH

📊 Neural Network Graph Built:
   Total Nodes: 929
   Total Edges: 981
   Transformer Layers: 28
   Attention Heads: 896

📋 Node Types Distribution:
type
attention_head    896
transformer        28
input               1
embedding           1
output              1
normalization       1
feedforward         1
Name: count, dtype: int64


In [9]:
# ==============================================================================
# Step 7: Initialize RAPIDS on GPU 1
# ==============================================================================

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print("="*70)
print("🔥 INITIALIZING RAPIDS ON GPU 1")
print("="*70)

import cudf
import cupy as cp
import cugraph

print(f"\n📊 RAPIDS GPU Info:")
device = cp.cuda.Device(0)  # Device 0 in filtered view = actual GPU 1
print(f"   Device: {device.id} (filtered view)")
print(f"   Actual GPU: 1 (Tesla T4)")
print(f"   Memory: {device.mem_info[1] / 1e9:.1f} GB free")

print(f"\n✅ RAPIDS initialized on GPU 1")

🔥 INITIALIZING RAPIDS ON GPU 1

📊 RAPIDS GPU Info:
   Device: 0 (filtered view)
   Actual GPU: 1 (Tesla T4)
   Memory: 15.8 GB free

✅ RAPIDS initialized on GPU 1


In [10]:
# ==============================================================================
# Step 8: GPU-Accelerated Graph Analytics
# ==============================================================================

print("="*70)
print("🔬 GPU-ACCELERATED GRAPH ANALYTICS")
print("="*70)

# Convert to cuDF for GPU processing
edges_cudf = cudf.DataFrame(edges_df)
nodes_cudf = cudf.DataFrame(nodes_df[['node_id']])  # Minimal for graph

# Create cuGraph
G = cugraph.Graph()
G.from_cudf_edgelist(edges_cudf, source='source', destination='target', edge_attr='weight')

print("\n📊 Graph Statistics:")
print(f"   Number of vertices: {G.number_of_vertices()}")
print(f"   Number of edges: {G.number_of_edges()}")
print(f"   Directed: {G.is_directed()}")

# PageRank - Identify important components
print("\n📊 PageRank Analysis (Component Importance):")
pagerank = cugraph.pagerank(G)
pagerank = pagerank.sort_values('pagerank', ascending=False)

# Merge PageRank back to nodes
pagerank_pd = pagerank.to_pandas().rename(columns={'vertex': 'node_id', 'pagerank': 'importance'})
nodes_df = nodes_df.merge(pagerank_pd, on='node_id', how='left')
nodes_df['importance'] = nodes_df['importance'].fillna(nodes_df['importance'].mean())

# Betweenness Centrality - Identify critical connections
print("\n📊 Betweenness Centrality (Critical Connections):")
bc = cugraph.betweenness_centrality(G)
bc_pd = bc.to_pandas().rename(columns={'vertex': 'node_id', 'betweenness_centrality': 'centrality'})
nodes_df = nodes_df.merge(bc_pd, on='node_id', how='left')
nodes_df['centrality'] = nodes_df['centrality'].fillna(0)

# Degree Centrality
print("\n📊 Degree Centrality (Connectivity):")
degree_df = cudf.DataFrame({
    'node_id': cudf.Series(range(G.number_of_vertices())),
})

# Calculate in/out degree
for i in range(G.number_of_vertices()):
    # Simplified degree calculation
    pass

print("\n✅ Graph analytics computed on GPU 1")

🔬 GPU-ACCELERATED GRAPH ANALYTICS

📊 Graph Statistics:
   Number of vertices: 928
   Number of edges: 981
   Directed: False

📊 PageRank Analysis (Component Importance):


/usr/local/lib/python3.12/dist-packages/cugraph/link_analysis/pagerank.py:232: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)



📊 Betweenness Centrality (Critical Connections):

📊 Degree Centrality (Connectivity):

✅ Graph analytics computed on GPU 1


In [11]:
# ==============================================================================
# Step 9: Register Graphistry
# ==============================================================================

print("="*70)
print("🔐 REGISTERING GRAPHISTRY")
print("="*70)

from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    graphistry_key_id = user_secrets.get_secret("Graphistry_Personal_Key_ID")
    graphistry_secret = user_secrets.get_secret("Graphistry_Personal_Secret_Key")
    
    graphistry.register(
        api=3,
        protocol="https",
        server="hub.graphistry.com",
        personal_key_id=graphistry_key_id,
        personal_key_secret=graphistry_secret
    )
    print("✅ Graphistry registered successfully")
except Exception as e:
    print(f"⚠️ Graphistry registration failed: {e}")
    print("   Add secrets: Graphistry_Personal_Key_ID, Graphistry_Personal_Secret_Key")
    # Continue with offline mode for demonstration
    graphistry.register(api=3, protocol="https", server="hub.graphistry.com")

🔐 REGISTERING GRAPHISTRY
✅ Graphistry registered successfully


In [12]:
# ==============================================================================
# Step 10: Create Neural Network Visualization Dashboard
# ==============================================================================

print("="*70)
print("🎨 CREATING NEURAL NETWORK VISUALIZATION DASHBOARD")
print("="*70)

# Prepare enhanced node data
print("\n📊 Preparing visualization data...")

# Calculate component metrics
nodes_df['layer_norm'] = nodes_df['layer'].apply(lambda x: x if isinstance(x, int) and x >= 0 else -1)
nodes_df['size_scaled'] = np.log10(nodes_df.get('size_mb', 1) + 1) * 20

# Color coding by component type
type_colors = {
    'input': '#FF6B6B',        # Red
    'output': '#4ECDC4',       # Teal
    'embedding': '#FFD166',    # Yellow
    'transformer': '#06D6A0',  # Green
    'attention_head': '#118AB2', # Blue
    'normalization': '#EF476F', # Pink
    'feedforward': '#073B4C'   # Dark Blue
}

# Icon mapping
type_icons = {
    'input': 'sign-in-alt',
    'output': 'sign-out-alt',
    'embedding': 'layer-group',
    'transformer': 'microchip',
    'attention_head': 'eye',
    'normalization': 'balance-scale',
    'feedforward': 'arrows-alt-h'
}

# Create rich tooltips
def create_tooltip(row):
    tooltip = f"<b>{row['name']}</b><br>"
    tooltip += f"Type: {row['type']}<br>"
    if row['layer'] != 'all' and row['layer'] >= 0:
        tooltip += f"Layer: {row['layer']}<br>"
    if 'head' in row and not pd.isna(row['head']):
        tooltip += f"Head: {int(row['head'])}<br>"
    if 'parameters' in row and not pd.isna(row['parameters']):
        tooltip += f"Parameters: {row['parameters']:,}<br>"
    if 'size_mb' in row and not pd.isna(row['size_mb']):
        tooltip += f"Size: {row['size_mb']:.1f} MB<br>"
    if 'importance' in row and not pd.isna(row['importance']):
        tooltip += f"Importance: {row['importance']:.4f}<br>"
    if 'centrality' in row and not pd.isna(row['centrality']):
        tooltip += f"Centrality: {row['centrality']:.4f}<br>"
    return tooltip

nodes_df['tooltip'] = nodes_df.apply(create_tooltip, axis=1)

# Edge tooltips
edges_df['edge_tooltip'] = edges_df.apply(
    lambda row: f"<b>{row['type']}</b><br>Weight: {row['weight']}", axis=1
)

print(f"📊 Graph Summary:")
print(f"   Nodes: {len(nodes_df)} neural network components")
print(f"   Edges: {len(edges_df)} connections")
print(f"   Component Types: {len(type_colors)} distinct types")

🎨 CREATING NEURAL NETWORK VISUALIZATION DASHBOARD

📊 Preparing visualization data...
📊 Graph Summary:
   Nodes: 929 neural network components
   Edges: 981 connections
   Component Types: 7 distinct types


In [13]:
# ==============================================================================
# Step 11: SIMPLER VERSION - Create Main Architecture Visualization
# ==============================================================================

print("\n🎨 Creating main architecture visualization (simpler version)...")

try:
    # 1. Apply colors directly to DataFrame columns
    nodes_df['color'] = nodes_df['type'].map(type_colors).fillna('#95A5A6')
    nodes_df['icon'] = nodes_df['type'].map(type_icons).fillna('circle')
    
    # 2. Apply edge colors
    edge_color_map = {'contains': '#BDC3C7', 'feeds_into': '#3498DB', 'uses': '#E74C3C'}
    edges_df['edge_color'] = edges_df['type'].map(edge_color_map).fillna('#CCCCCC')
    
    # 3. Scale importance for point size (0-1 range)
    if nodes_df['importance'].max() > nodes_df['importance'].min():
        nodes_df['point_size'] = 15 + (nodes_df['importance'] - nodes_df['importance'].min()) / \
                                 (nodes_df['importance'].max() - nodes_df['importance'].min()) * 65
    else:
        nodes_df['point_size'] = 40
    
    # 4. Create the graph with direct binding
    g = graphistry.bind(
        source='source',
        destination='target',
        node='node_id',
        point_title='tooltip',
        point_color='color',
        point_size='point_size',
        point_icon='icon',
        edge_title='edge_tooltip',
        edge_color='edge_color',
        edge_weight='weight'
    )
    
    # 5. Apply settings
    g = g.settings(url_params={
        'play': 0,
        'pointSize': 2.5,
        'edgeOpacity': 0.6,
        'bg': '%23FFFFFF',
        'strongGravity': 'true',
        'edgeInfluence': 1.0,
        'scalingRatio': 10.0
    })
    
    # 6. Create visualization
    plotter = g.edges(edges_df).nodes(nodes_df)
    
    main_url = plotter.plot(
        render=False,
        name=f"GGUF Neural Network Architecture - {MODEL_FILE}",
        description=f"Visualization of {MODEL_FILE} with {n_layers} layers, {n_heads} heads per layer"
    )
    
    print(f"\n🚀 Main Visualization Created!")
    print(f"🔗 URL: {main_url}")
    
    from IPython.display import display, HTML
    display(HTML(
        f'<div style="margin:20px; padding:20px; background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); '
        f'border-radius:12px; color:white; box-shadow:0 4px 6px rgba(0,0,0,0.1);">'
        f'<h3 style="margin:0 0 10px 0;">🧠 GGUF Neural Network Architecture</h3>'
        f'<p style="margin:5px 0;">Interactive visualization of {MODEL_FILE}</p>'
        f'<a href="{main_url}" target="_blank" style="display:inline-block; margin-top:15px; padding:12px 24px; '
        f'background:white; color:#667eea; text-decoration:none; border-radius:6px; font-weight:bold; '
        f'box-shadow:0 2px 4px rgba(0,0,0,0.1);">🚀 Open Main Visualization</a>'
        f'</div>'
    ))
    
except Exception as e:
    print(f"❌ Visualization error: {e}")
    import traceback
    traceback.print_exc()


🎨 Creating main architecture visualization (simpler version)...


/usr/local/lib/python3.12/dist-packages/graphistry/util.py:241: RuntimeWarning: Coerced mixed-type columns to string for Arrow conversion: ['layer']. Convert explicitly before plot() for better control.
  warnings.warn(RuntimeWarning(msg))



🚀 Main Visualization Created!
🔗 URL: https://hub.graphistry.com/graph/graph.html?dataset=db33bdcae73d4420bc97304709921d57&type=arrow&viztoken=43b3b95a-e6ae-4d67-944d-73b810b7541a&usertag=438152b6-pygraphistry-0.50.4&splashAfter=1769194757&info=true&play=0&pointSize=2.5&edgeOpacity=0.6&bg=%23FFFFFF&strongGravity=true&edgeInfluence=1.0&scalingRatio=10.0


In [14]:
# ==============================================================================
# Step 12: Create Layer-by-Layer Subgraph Visualizations (FIXED)
# ==============================================================================

print("\n🔍 Creating layer-by-layer visualizations...")

layer_urls = {}

for layer_num in range(1, min(n_layers + 1, 6)):  # First 5 layers only
    # Filter nodes for this layer
    layer_nodes = nodes_df[
        (nodes_df['layer'] == layer_num) | 
        (nodes_df['layer'] == 'all') |
        (nodes_df['name'].str.contains(f'L{layer_num}_'))
    ].copy()  # Use copy() to avoid SettingWithCopyWarning
    
    if len(layer_nodes) > 0:
        # Scale importance for point size (0-1 range) - same as Step 11
        if layer_nodes['importance'].max() > layer_nodes['importance'].min():
            layer_nodes['point_size'] = 20 + (layer_nodes['importance'] - layer_nodes['importance'].min()) / \
                                     (layer_nodes['importance'].max() - layer_nodes['importance'].min()) * 40
        else:
            layer_nodes['point_size'] = 40
        
        # Apply colors and icons
        layer_nodes['color'] = layer_nodes['type'].map(type_colors).fillna('#95A5A6')
        layer_nodes['icon'] = layer_nodes['type'].map(type_icons).fillna('circle')
    
    layer_node_ids = layer_nodes['node_id'].tolist()
    
    # Filter edges connecting these nodes
    layer_edges = edges_df[
        edges_df['source'].isin(layer_node_ids) & 
        edges_df['target'].isin(layer_node_ids)
    ]
    
    if len(layer_nodes) > 0 and len(layer_edges) > 0:
        # Create layer-specific visualization with direct binding like Step 11
        layer_g = graphistry.bind(
            source='source',
            destination='target',
            node='node_id',
            point_title='name',
            point_color='color',
            point_size='point_size',
            point_icon='icon'
        )
        
        layer_plotter = layer_g.edges(layer_edges).nodes(layer_nodes)
        
        # Apply settings
        layer_plotter = layer_plotter.settings(url_params={
            'play': 0,
            'pointSize': 3.0,
            'edgeOpacity': 0.7,
            'bg': '%23F8F9FA',
            'strongGravity': 'true',
            'edgeInfluence': 1.0,
            'scalingRatio': 8.0,
            'showLabels': True
        })
        
        try:
            layer_url = layer_plotter.plot(
                render=False,
                name=f"Layer {layer_num} - {MODEL_FILE}",
                description=f"Detailed view of transformer layer {layer_num}"
            )
            layer_urls[f"Layer {layer_num}"] = layer_url
            print(f"   ✅ Layer {layer_num}: {len(layer_nodes)} nodes, {len(layer_edges)} edges")
        except Exception as e:
            print(f"   ⚠️ Layer {layer_num} visualization failed: {e}")


🔍 Creating layer-by-layer visualizations...


/usr/local/lib/python3.12/dist-packages/graphistry/util.py:241: RuntimeWarning: Coerced mixed-type columns to string for Arrow conversion: ['layer']. Convert explicitly before plot() for better control.
  warnings.warn(RuntimeWarning(msg))


   ✅ Layer 1: 35 nodes, 34 edges


/usr/local/lib/python3.12/dist-packages/graphistry/util.py:241: RuntimeWarning: Coerced mixed-type columns to string for Arrow conversion: ['layer']. Convert explicitly before plot() for better control.
  warnings.warn(RuntimeWarning(msg))


   ✅ Layer 2: 35 nodes, 34 edges


...

   ✅ Layer 3: 35 nodes, 34 edges


/usr/local/lib/python3.12/dist-packages/graphistry/util.py:241: RuntimeWarning: Coerced mixed-type columns to string for Arrow conversion: ['layer']. Convert explicitly before plot() for better control.
  warnings.warn(RuntimeWarning(msg))


   ✅ Layer 4: 35 nodes, 34 edges


/usr/local/lib/python3.12/dist-packages/graphistry/util.py:241: RuntimeWarning: Coerced mixed-type columns to string for Arrow conversion: ['layer']. Convert explicitly before plot() for better control.
  warnings.warn(RuntimeWarning(msg))


   ✅ Layer 5: 35 nodes, 34 edges


In [15]:
# ==============================================================================
# Step 12b: Create Interactive Layer Switcher Visualization
# ==============================================================================

print("\n🔄 Creating interactive layer switcher visualization...")

try:
    # Add a 'layer_group' column for filtering
    nodes_df['layer_group'] = nodes_df['layer'].astype(str)
    
    # Create a unified visualization with layer filtering
    interactive_g = graphistry.bind(
        source='source',
        destination='target',
        node='node_id',
        point_title='name',
        point_color='color',
        point_size='point_size',
        point_icon='icon',
        point_label='layer_group'  # Use for filtering
    )
    
    # Create a combined visualization with all layers
    interactive_plotter = interactive_g.edges(edges_df).nodes(nodes_df)
    
    # Add filter controls for layers
    interactive_plotter = interactive_plotter.settings(url_params={
        'play': 0,
        'pointSize': 2.5,
        'edgeOpacity': 0.6,
        'bg': '%23FFFFFF',
        'strongGravity': 'true',
        'edgeInfluence': 1.0,
        'scalingRatio': 10.0,
        'showFilters': 'true',  # Enable filters panel
        'showLabels': 'true',
        'sidebarMode': 'full'  # Show full sidebar with filters
    })
    
    interactive_url = interactive_plotter.plot(
        render=False,
        name=f"Interactive Layers - {MODEL_FILE}",
        description=f"Interactive visualization of {MODEL_FILE} with layer filtering"
    )
    
    print(f"\n🚀 Interactive Layer Switcher Created!")
    print(f"🔗 URL: {interactive_url}")
    
    from IPython.display import display, HTML
    display(HTML(
        f'<div style="margin:20px; padding:20px; background:linear-gradient(135deg, #00b09b 0%, #96c93d 100%); '
        f'border-radius:12px; color:white; box-shadow:0 4px 6px rgba(0,0,0,0.1);">'
        f'<h3 style="margin:0 0 10px 0;">🔀 Interactive Layer Explorer</h3>'
        f'<p style="margin:5px 0;">Filter layers using the sidebar controls in Graphistry</p>'
        f'<ul style="margin:10px 0 15px 0; padding-left:20px;">'
        f'<li>Use the <strong>Filters panel</strong> on the right</li>'
        f'<li>Filter by <strong>layer_group</strong> to show specific layers</li>'
        f'<li>Click nodes to see detailed information</li>'
        f'</ul>'
        f'<a href="{interactive_url}" target="_blank" style="display:inline-block; margin-top:15px; padding:12px 24px; '
        f'background:white; color:#00b09b; text-decoration:none; border-radius:6px; font-weight:bold; '
        f'box-shadow:0 2px 4px rgba(0,0,0,0.1);">🔀 Open Interactive Explorer</a>'
        f'</div>'
    ))
    
except Exception as e:
    print(f"❌ Interactive visualization error: {e}")
    import traceback
    traceback.print_exc()


🔄 Creating interactive layer switcher visualization...


/usr/local/lib/python3.12/dist-packages/graphistry/util.py:241: RuntimeWarning: Coerced mixed-type columns to string for Arrow conversion: ['layer']. Convert explicitly before plot() for better control.
  warnings.warn(RuntimeWarning(msg))



🚀 Interactive Layer Switcher Created!
🔗 URL: https://hub.graphistry.com/graph/graph.html?dataset=0b3fa715857d463cb9bf3f4ceedbc86a&type=arrow&viztoken=167362cd-161e-4fef-922f-b332970f5628&usertag=438152b6-pygraphistry-0.50.4&splashAfter=1769194779&info=true&play=0&pointSize=2.5&edgeOpacity=0.6&bg=%23FFFFFF&strongGravity=true&edgeInfluence=1.0&scalingRatio=10.0&showFilters=true&showLabels=true&sidebarMode=full


In [24]:
# ==============================================================================
# Step 13: Create Attention Head Visualization (FIXED)
# ==============================================================================

print("\n👁️ Creating attention head visualization...")

# Focus on attention heads
attention_nodes = nodes_df[nodes_df['type'] == 'attention_head']
if len(attention_nodes) > 0:
    # Get first layer's attention heads
    first_layer_heads = attention_nodes[attention_nodes['layer'] == 1].copy()
    
    if len(first_layer_heads) > 0:
        # Find the layer node ID for layer 1
        # In your graph, layer nodes have type 'transformer' and layer = 1
        layer_1_node = nodes_df[(nodes_df['type'] == 'transformer') & (nodes_df['layer'] == 1)]
        
        if len(layer_1_node) > 0:
            layer_1_id = layer_1_node.iloc[0]['node_id']
            print(f"   Found Layer 1 node ID: {layer_1_id}")
        else:
            # If not found, use the first node that contains "Layer_1" in name
            layer_1_node = nodes_df[nodes_df['name'].str.contains('Layer_1')]
            if len(layer_1_node) > 0:
                layer_1_id = layer_1_node.iloc[0]['node_id']
                print(f"   Found Layer 1 node ID (by name): {layer_1_id}")
            else:
                print("   ⚠️ Could not find Layer 1 node")
                layer_1_id = None
        
        # Pre-process data in DataFrame
        if len(first_layer_heads['head'].unique()) > 1:
            # Create a continuous color mapping for heads
            unique_heads = sorted(first_layer_heads['head'].unique())
            head_palette = ['#FF6B6B', '#4ECDC4', '#FFD166', '#95E1D3', '#F38181', '#A8D8EA', 
                           '#AA96DA', '#FCBAD3', '#FFFFD2', '#A8E6CF', '#DCEDC1', '#FFD3B6',
                           '#FFAAA5', '#FF8B94', '#CCF6C8', '#F9F3DF', '#CDF0EA', '#FAEEE7']
            
            # Map head numbers to colors
            head_color_map = {}
            for i, head_num in enumerate(unique_heads):
                color_idx = i % len(head_palette)
                head_color_map[head_num] = head_palette[color_idx]
            
            first_layer_heads['color'] = first_layer_heads['head'].map(head_color_map)
        else:
            first_layer_heads['color'] = '#667eea'  # Default color
        
        # Scale importance for point size
        if first_layer_heads['importance'].max() > first_layer_heads['importance'].min():
            first_layer_heads['point_size'] = 25 + (
                (first_layer_heads['importance'] - first_layer_heads['importance'].min()) / 
                (first_layer_heads['importance'].max() - first_layer_heads['importance'].min()) * 25
            )
        else:
            first_layer_heads['point_size'] = 40
        
        # Add tooltip
        first_layer_heads['tooltip'] = first_layer_heads.apply(
            lambda row: f"<b>Head {int(row['head'])}</b><br>"
                       f"Layer: {row['layer']}<br>"
                       f"Importance: {row['importance']:.4f}<br>"
                       f"Parameters: ~{row.get('parameters', 'N/A'):,}", 
            axis=1
        )
        
        head_ids = first_layer_heads['node_id'].tolist()
        
        # Find edges between these heads and their layer
        if layer_1_id:
            # Look for edges where attention heads are connected to their layer
            head_edges = edges_df[
                ((edges_df['source'].isin(head_ids)) & (edges_df['target'] == layer_1_id)) |
                ((edges_df['target'].isin(head_ids)) & (edges_df['source'] == layer_1_id)) |
                ((edges_df['source'].isin(head_ids)) & edges_df['target'].isin(head_ids))  # Connections between heads
            ]
            
            if len(head_edges) == 0:
                print("   ⚠️ No edges found between attention heads and layer")
                print("   Creating artificial edges for visualization...")
                
                # Create artificial edges connecting heads to layer
                artificial_edges = []
                for head_id in head_ids:
                    # Connect each head to the layer
                    artificial_edges.append({
                        'source': layer_1_id,
                        'target': head_id,
                        'type': 'contains',
                        'weight': 1.0,
                        'edge_tooltip': f'Layer 1 → Head {head_ids.index(head_id)}'
                    })
                    
                    # Connect heads in a ring for better visualization
                    if len(head_ids) > 1:
                        next_head_idx = (head_ids.index(head_id) + 1) % len(head_ids)
                        artificial_edges.append({
                            'source': head_id,
                            'target': head_ids[next_head_idx],
                            'type': 'attention_flow',
                            'weight': 0.3,
                            'edge_tooltip': f'Attention flow between heads'
                        })
                
                head_edges = pd.DataFrame(artificial_edges)
                print(f"   Created {len(head_edges)} artificial edges")
        else:
            print("   ⚠️ No layer ID found, showing heads without connections")
            head_edges = pd.DataFrame(columns=edges_df.columns)
        
        # Add the layer node to the visualization
        if layer_1_id:
            layer_node_data = layer_1_node.copy()
            layer_node_data['color'] = '#06D6A0'  # Green for transformer
            layer_node_data['point_size'] = 50
            layer_node_data['tooltip'] = f"<b>Layer 1</b><br>Type: transformer<br>Parameters: ~{layer_node_data.iloc[0].get('parameters', 'N/A'):,}"
            
            # Combine layer node with heads
            all_nodes = pd.concat([first_layer_heads, layer_node_data], ignore_index=True)
        else:
            all_nodes = first_layer_heads
        
        heads_g = graphistry.bind(
            source='source',
            destination='target',
            node='node_id',
            point_title='tooltip',
            point_color='color',
            point_size='point_size',
            point_label='name'
        )
        
        heads_plotter = heads_g.edges(head_edges).nodes(all_nodes)
        
        # Apply settings
        heads_plotter = heads_plotter.settings(url_params={
            'play': 0,
            'pointSize': 3.5,
            'edgeOpacity': 0.8,
            'bg': '%23FFFFFF',
            'layout': 'concentric',
            'strongGravity': 'true',
            'edgeInfluence': 1.0,
            'scalingRatio': 6.0,
            'showLabels': 'true',
            'gravity': 0.1,
            'linkDistance': 100
        })
        
        try:
            heads_url = heads_plotter.plot(
                render=False,
                name=f"Attention Heads - {MODEL_FILE}",
                description=f"Visualization of {len(first_layer_heads)} attention heads in layer 1"
            )
            layer_urls["Attention Heads"] = heads_url
            print(f"   ✅ Attention Heads: {len(first_layer_heads)} heads visualized")
            print(f"   🔗 URL: {heads_url}")

            from IPython.display import display, HTML
            display(HTML(
                f'<div style="margin:20px; padding:20px; background:linear-gradient(135deg, #FF6B6B 0%, #FFD166 100%); '
                f'border-radius:12px; color:white; box-shadow:0 4px 6px rgba(0,0,0,0.1);">'
                f'<h3 style="margin:0 0 10px 0;">👁️ Attention Heads Visualization</h3>'
                f'<p style="margin:5px 0;">Interactive visualization of {len(first_layer_heads)} attention heads in Layer 1</p>'
                f'<ul style="margin:10px 0 15px 0; padding-left:20px;">'
                f'<li><strong>Layer 1</strong>: {len(first_layer_heads)} attention heads</li>'
                f'<li>Color-coded by head number</li>'
                f'<li>Central transformer layer node in green</li>'
                f'</ul>'
                f'<a href="{heads_url}" target="_blank" style="display:inline-block; margin-top:15px; padding:12px 24px; '
                f'background:white; color:#FF6B6B; text-decoration:none; border-radius:6px; font-weight:bold; '
                f'box-shadow:0 2px 4px rgba(0,0,0,0.1);">👁️ Open Attention Heads Visualization</a>'
                f'</div>'
            ))
        except Exception as e:
            print(f"   ⚠️ Attention heads visualization failed: {e}")
            import traceback
            traceback.print_exc()
else:
    print("   ⚠️ No attention head nodes found in the data")


👁️ Creating attention head visualization...
   Found Layer 1 node ID: 3
   ✅ Attention Heads: 32 heads visualized
   🔗 URL: https://hub.graphistry.com/graph/graph.html?dataset=24da5c1158324e69940716b6d95e9fe5&type=arrow&viztoken=28009142-f96e-4e3a-b2ed-ed8a058575bd&usertag=438152b6-pygraphistry-0.50.4&splashAfter=1769196721&info=true&play=0&pointSize=3.5&edgeOpacity=0.8&bg=%23FFFFFF&layout=concentric&strongGravity=true&edgeInfluence=1.0&scalingRatio=6.0&showLabels=true&gravity=0.1&linkDistance=100


In [18]:
# ==============================================================================
# Step 13b: Create and Display Attention Head Dashboard (FIXED for Kaggle)
# ==============================================================================

print("\n📊 Creating attention head dashboard...")

try:
    # Check if we have attention heads
    if len(attention_nodes) > 0:
        # Calculate statistics
        total_heads = len(attention_nodes)
        head_layers = attention_nodes['layer'].nunique()
        avg_heads_per_layer = total_heads / head_layers if head_layers > 0 else 0
        max_importance = attention_nodes['importance'].max() if total_heads > 0 else 0
        
        # Create a simpler HTML dashboard that can be displayed inline
        dashboard_html = f'''
        <!DOCTYPE html>
        <html>
        <head>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 20px;
                    background: #f5f7fa;
                    color: #333;
                }}
                .dashboard {{
                    max-width: 1200px;
                    margin: 0 auto;
                }}
                .header {{
                    background: linear-gradient(135deg, #FF6B6B 0%, #4ECDC4 100%);
                    color: white;
                    padding: 25px;
                    border-radius: 12px;
                    margin-bottom: 25px;
                    text-align: center;
                }}
                .stats-container {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
                    gap: 20px;
                    margin-bottom: 30px;
                }}
                .stat-card {{
                    background: white;
                    padding: 20px;
                    border-radius: 10px;
                    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
                    text-align: center;
                }}
                .stat-value {{
                    font-size: 36px;
                    font-weight: bold;
                    color: #2c5364;
                    margin: 10px 0;
                }}
                .stat-label {{
                    color: #666;
                    font-size: 14px;
                    text-transform: uppercase;
                }}
                .btn-container {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
                    gap: 15px;
                    margin: 25px 0;
                }}
                .btn {{
                    display: block;
                    padding: 18px;
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white;
                    text-decoration: none;
                    border-radius: 8px;
                    font-weight: bold;
                    text-align: center;
                    font-size: 16px;
                    transition: transform 0.3s;
                }}
                .btn:hover {{
                    transform: translateY(-2px);
                    box-shadow: 0 6px 12px rgba(0,0,0,0.15);
                }}
                .btn.attention {{
                    background: linear-gradient(135deg, #FF6B6B 0%, #FFD166 100%);
                }}
                .btn.full {{
                    background: linear-gradient(135deg, #00b09b 0%, #96c93d 100%);
                }}
                .info-box {{
                    background: #e8f4fc;
                    border-left: 4px solid #4ECDC4;
                    padding: 15px;
                    margin: 20px 0;
                    border-radius: 0 8px 8px 0;
                }}
                .color-legend {{
                    display: flex;
                    flex-wrap: wrap;
                    gap: 15px;
                    margin: 15px 0;
                    padding: 15px;
                    background: white;
                    border-radius: 8px;
                    border: 1px solid #eee;
                }}
                .color-item {{
                    display: flex;
                    align-items: center;
                    gap: 8px;
                }}
                .color-dot {{
                    width: 16px;
                    height: 16px;
                    border-radius: 50%;
                }}
                .layer-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fill, minmax(150px, 1fr));
                    gap: 10px;
                    margin: 20px 0;
                }}
                .layer-btn {{
                    padding: 12px;
                    background: #f8f9fa;
                    border: 1px solid #dee2e6;
                    border-radius: 6px;
                    text-align: center;
                    text-decoration: none;
                    color: #495057;
                    transition: all 0.2s;
                }}
                .layer-btn:hover {{
                    background: #667eea;
                    color: white;
                    border-color: #667eea;
                }}
            </style>
        </head>
        <body>
            <div class="dashboard">
                <div class="header">
                    <h1 style="margin: 0; font-size: 32px;">🧠 Attention Head Dashboard</h1>
                    <p style="margin: 10px 0 0 0; font-size: 16px; opacity: 0.9;">
                        Model: {MODEL_FILE} • {pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")}
                    </p>
                </div>
                
                <div class="stats-container">
                    <div class="stat-card">
                        <div class="stat-label">Total Attention Heads</div>
                        <div class="stat-value">{total_heads}</div>
                        <p>Across all layers</p>
                    </div>
                    <div class="stat-card">
                        <div class="stat-label">Layers with Heads</div>
                        <div class="stat-value">{head_layers}</div>
                        <p>Layers containing attention</p>
                    </div>
                    <div class="stat-card">
                        <div class="stat-label">Avg Heads/Layer</div>
                        <div class="stat-value">{avg_heads_per_layer:.1f}</div>
                        <p>Average per layer</p>
                    </div>
                    <div class="stat-card">
                        <div class="stat-label">Max Importance</div>
                        <div class="stat-value">{max_importance:.3f}</div>
                        <p>Highest attention weight</p>
                    </div>
                </div>
                
                <div class="info-box">
                    <p><strong>📊 Dashboard Overview:</strong> This dashboard provides insights into the attention mechanism of your transformer model. Attention heads are the core components that allow the model to focus on different parts of the input sequence.</p>
                </div>
                
                <h2>🔗 Quick Links</h2>
                <div class="btn-container">
                    <a href="{heads_url}" class="btn attention" target="_blank">
                        🎯 Attention Heads Visualization
                    </a>
                    <a href="{main_url}" class="btn" target="_blank">
                        🌐 Full Architecture
                    </a>
        '''
        
        # Add interactive URL if available
        if 'interactive_url' in locals():
            dashboard_html += f'''
                    <a href="{interactive_url}" class="btn full" target="_blank">
                        🔄 Interactive Explorer
                    </a>
            '''
        
        dashboard_html += '''
                </div>
                
                <h2>🎨 Layer Visualizations</h2>
                <div class="layer-grid">
        '''
        
        # Add layer buttons (first 5)
        for i in range(1, min(6, n_layers + 1)):
            if f"Layer {i}" in layer_urls:
                dashboard_html += f'''
                    <a href="{layer_urls[f'Layer {i}']}" class="layer-btn" target="_blank">
                        Layer {i}
                    </a>
                '''
        
        dashboard_html += '''
                </div>
                
                <div class="info-box">
                    <p><strong>💡 How to use:</strong> Click on any visualization link to open it in a new tab. Use the Graphistry interface to explore, zoom, and interact with the neural network structure.</p>
                </div>
            </div>
        </body>
        </html>
        '''
        
        # Display the dashboard directly in the notebook
        from IPython.display import display, HTML
        display(HTML(dashboard_html))
        
        # Also save to file
        attention_dashboard_path = '/kaggle/working/attention_dashboard.html'
        with open(attention_dashboard_path, 'w') as f:
            f.write(dashboard_html)
        
        print(f"\n✅ Dashboard created and displayed above")
        print(f"📁 Dashboard also saved to: {attention_dashboard_path}")
        
        # Provide direct download link
        print(f"\n📥 To download the dashboard HTML file:")
        print(f"   1. Click on the 'Data' tab in Kaggle")
        print(f"   2. Navigate to '/kaggle/working/'")
        print(f"   3. Download 'attention_dashboard.html'")
        
    else:
        print("   ⚠️ No attention head nodes available for dashboard creation")
        
except Exception as e:
    print(f"❌ Dashboard creation error: {e}")
    import traceback
    traceback.print_exc()


📊 Creating attention head dashboard...



✅ Dashboard created and displayed above
📁 Dashboard also saved to: /kaggle/working/attention_dashboard.html

📥 To download the dashboard HTML file:
   1. Click on the 'Data' tab in Kaggle
   2. Navigate to '/kaggle/working/'
   3. Download 'attention_dashboard.html'


In [19]:
# ==============================================================================
# Step 14: Create Quantization Block Visualization (FIXED v2)
# ==============================================================================

print("\n⚖️ Creating quantization block visualization...")

# Simulate quantization blocks (Q4_K_M has specific block structure)
quantization_blocks = []
block_id = 1000  # Starting ID for quantization blocks

# GGUF Q4_K_M uses blocks of weights
for layer in range(1, n_layers + 1):
    for block in range(4):  # 4 quantization blocks per layer (simplified)
        block_node = {
            'node_id': block_id,
            'name': f'QBlock_L{layer}_{block}',
            'type': 'quantization',
            'layer': layer,
            'block': block,
            'parameters': hidden_dim * hidden_dim // 16,  # Approx for Q4
            'size_mb': (hidden_dim * hidden_dim * 0.5) / (1024**2)  # 0.5 bytes per param for Q4
        }
        quantization_blocks.append(block_node)
        block_id += 1

if quantization_blocks:
    quant_df = pd.DataFrame(quantization_blocks).copy()  # Use copy()
    
    # Pre-process data in DataFrame
    # Create color mapping for layers
    unique_layers = sorted(quant_df['layer'].unique())
    layer_palette = ['#FF6B6B', '#4ECDC4', '#FFD166', '#95E1D3', '#F38181', '#A8D8EA', '#C86B85', '#6B8CFF']
    
    # Map layer numbers to colors
    layer_color_map = {}
    for i, layer_num in enumerate(unique_layers):
        color_idx = i % len(layer_palette)
        layer_color_map[layer_num] = layer_palette[color_idx]
    
    quant_df['color'] = quant_df['layer'].map(layer_color_map)
    
    # Scale size_mb for point size
    if quant_df['size_mb'].max() > quant_df['size_mb'].min():
        quant_df['point_size'] = 20 + (
            (quant_df['size_mb'] - quant_df['size_mb'].min()) / 
            (quant_df['size_mb'].max() - quant_df['size_mb'].min()) * 40
        )
    else:
        quant_df['point_size'] = 40
    
    # Add tooltip
    quant_df['tooltip'] = quant_df.apply(
        lambda row: f"Quant Block L{row['layer']}.{row['block']}<br>"
                   f"Size: {row['size_mb']:.2f} MB<br>"
                   f"Params: {row['parameters']:,}<br>"
                   f"Type: {row['type']}", 
        axis=1
    )
    
    # Connect quantization blocks to their layers
    quant_edges = []
    for _, block in quant_df.iterrows():
        # Try to find the corresponding layer node
        # Look for layer nodes (assuming they have IDs like 'L1', 'L2' or numeric IDs)
        layer_pattern = f"L{int(block['layer'])}"
        layer_nodes = nodes_df[nodes_df['name'].str.contains(layer_pattern, na=False)]
        
        if len(layer_nodes) > 0:
            layer_id = layer_nodes.iloc[0]['node_id']
        else:
            # Fallback: use layer number as ID
            layer_id = int(block['layer'])
        
        quant_edges.append({
            'source': layer_id,
            'target': int(block['node_id']),
            'type': 'quantizes',
            'weight': 0.8,
            'edge_tooltip': f"Layer {int(block['layer'])} → Quant Block {int(block['block'])}",
            'edge_color': '#9B59B6'  # Add color as column in edges DataFrame
        })
    
    quant_edges_df = pd.DataFrame(quant_edges)
    
    # Create visualization - IMPORTANT: edge_color now refers to column name
    quant_g = graphistry.bind(
        source='source',
        destination='target',
        node='node_id',
        point_title='tooltip',
        point_color='color',
        point_size='point_size',
        point_label='name',
        edge_title='edge_tooltip',
        edge_color='edge_color'  # This now refers to the column we added
    )
    
    quant_plotter = quant_g.edges(quant_edges_df).nodes(quant_df)
    
    # Apply settings
    quant_plotter = quant_plotter.settings(url_params={
        'play': 0,
        'pointSize': 3.0,
        'edgeOpacity': 0.6,
        'bg': '%23F0F4FF',
        'layout': 'grid',
        'strongGravity': 'true',
        'edgeInfluence': 1.2,
        'scalingRatio': 5.0,
        'showLabels': 'true'
    })
    
    try:
        quant_url = quant_plotter.plot(
            render=False,
            name=f"Quantization Blocks - {MODEL_FILE}",
            description=f"Q4_K_M quantization blocks across {n_layers} layers"
        )
        layer_urls["Quantization Blocks"] = quant_url
        print(f"   ✅ Quantization Blocks: {len(quant_df)} blocks visualized")
        print(f"   📊 Block size range: {quant_df['size_mb'].min():.2f} - {quant_df['size_mb'].max():.2f} MB per block")
        print(f"   🔗 Total quantization blocks: {len(quant_df)} across {n_layers} layers")
    except Exception as e:
        print(f"   ⚠️ Quantization visualization failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("   ⚠️ No quantization blocks created")


⚖️ Creating quantization block visualization...
   ✅ Quantization Blocks: 112 blocks visualized
   📊 Block size range: 4.50 - 4.50 MB per block
   🔗 Total quantization blocks: 112 across 28 layers


In [20]:
# ==============================================================================
# Step 15: Create Interactive Dashboard (FIXED)
# ==============================================================================

print("="*70)
print("📊 CREATING INTERACTIVE DASHBOARD")
print("="*70)

# Collect all visualization URLs
all_visualizations = {"Main Architecture": main_url}
all_visualizations.update(layer_urls)

# Add quantization visualization if it exists
if "Quantization Blocks" in layer_urls:
    all_visualizations["Quantization Blocks"] = layer_urls["Quantization Blocks"]

# Count attention heads
attention_heads_count = len(nodes_df[nodes_df['type'] == 'attention_head']) if 'nodes_df' in locals() else 0
total_nodes = len(nodes_df) if 'nodes_df' in locals() else 0
total_edges = len(edges_df) if 'edges_df' in locals() else 0

# Define description function
def get_viz_description(viz_name):
    descriptions = {
        "Main Architecture": "Complete overview of the entire neural network architecture with all layers and connections.",
        "Interactive Explorer": "Filter and explore different components interactively with sidebar controls.",
        "Attention Heads": "Visualization of multi-head attention mechanisms colored by head number.",
        "Quantization Blocks": "Q4_K_M quantization blocks showing memory distribution across layers.",
        "Layer 1": "Detailed view of transformer layer 1 with attention heads and feed-forward networks.",
        "Layer 2": "Detailed view of transformer layer 2 showing internal connections.",
        "Layer 3": "Detailed view of transformer layer 3 architecture and components.",
        "Layer 4": "Detailed view of transformer layer 4 structure and connections.",
        "Layer 5": "Detailed view of transformer layer 5 with component visualization.",
    }
    return descriptions.get(viz_name, f"Detailed visualization of {viz_name.lower()}.")

# Create dashboard HTML
dashboard_html = f'''
<div style="margin:25px 0; padding:30px; background:linear-gradient(135deg, #0f2027 0%, #203a43 50%, #2c5364 100%); border-radius:16px; box-shadow:0 20px 60px rgba(0,0,0,0.3); color:white;">
    <div style="text-align:center; margin-bottom:35px;">
        <h2 style="margin:0 0 10px 0; font-size:32px; color:white;">🧠 GGUF Neural Network Visualization Dashboard</h2>
        <p style="margin:0; color:rgba(255,255,255,0.8); font-size:16px;">Interactive visualizations of {MODEL_FILE} architecture</p>
    </div>
    
    <!-- Statistics Overview -->
    <div style="background:rgba(255,255,255,0.1); border-radius:12px; padding:25px; margin-bottom:30px; backdrop-filter:blur(10px);">
        <h3 style="margin:0 0 20px 0; color:white; text-align:center;">📊 Model Statistics</h3>
        <div style="display:grid; grid-template-columns:repeat(auto-fit, minmax(200px, 1fr)); gap:20px;">
            <div style="text-align:center;">
                <div style="font-size:14px; color:rgba(255,255,255,0.7); margin-bottom:8px;">Total Layers</div>
                <div style="font-size:36px; font-weight:700; color:#4ECDC4;">{n_layers}</div>
            </div>
            <div style="text-align:center;">
                <div style="font-size:14px; color:rgba(255,255,255,0.7); margin-bottom:8px;">Attention Heads</div>
                <div style="font-size:36px; font-weight:700; color:#FF6B6B;">{attention_heads_count}</div>
            </div>
            <div style="text-align:center;">
                <div style="font-size:14px; color:rgba(255,255,255,0.7); margin-bottom:8px;">Total Nodes</div>
                <div style="font-size:36px; font-weight:700; color:#FFD166;">{total_nodes}</div>
            </div>
            <div style="text-align:center;">
                <div style="font-size:14px; color:rgba(255,255,255,0.7); margin-bottom:8px;">Total Edges</div>
                <div style="font-size:36px; font-weight:700; color:#95E1D3;">{total_edges}</div>
            </div>
            <div style="text-align:center;">
                <div style="font-size:14px; color:rgba(255,255,255,0.7); margin-bottom:8px;">Heads/Layer</div>
                <div style="font-size:36px; font-weight:700; color:#A8D8EA;">{n_heads}</div>
            </div>
            <div style="text-align:center;">
                <div style="font-size:14px; color:rgba(255,255,255,0.7); margin-bottom:8px;">Quant Blocks</div>
                <div style="font-size:36px; font-weight:700; color:#9B59B6;">{len(quant_df) if 'quant_df' in locals() else 0}</div>
            </div>
        </div>
    </div>
    
    <!-- Visualization Grid -->
    <h3 style="margin:0 0 20px 0; color:white; text-align:center;">🚀 Available Visualizations</h3>
    <div style="display:grid; grid-template-columns:repeat(auto-fill, minmax(320px, 1fr)); gap:25px;">
'''

# Define category colors and icons
category_info = {
    "Main Architecture": {"color": "#667eea", "icon": "🌐"},
    "Interactive Explorer": {"color": "#10b981", "icon": "🔍"},
    "Attention Heads": {"color": "#FF6B6B", "icon": "🎯"},
    "Quantization Blocks": {"color": "#9B59B6", "icon": "⚖️"},
}

# Add cards for each visualization
for viz_name, viz_url in all_visualizations.items():
    if viz_url:
        # Determine category
        if viz_name == "Main Architecture":
            category = "Main Architecture"
        elif "Interactive" in viz_name or "Explorer" in viz_name:
            category = "Interactive Explorer"
        elif "Attention" in viz_name:
            category = "Attention Heads"
        elif "Quantization" in viz_name:
            category = "Quantization Blocks"
        else:
            category = "Layer Detail"
        
        # Get styling
        if category in category_info:
            accent_color = category_info[category]["color"]
            icon = category_info[category]["icon"]
        else:
            accent_color = "#6b7280"
            icon = "🔧"
        
        # Get description
        description = get_viz_description(viz_name)
        
        dashboard_html += f'''
        <div style="background:rgba(255,255,255,0.95); border-radius:12px; padding:25px; box-shadow:0 8px 25px rgba(0,0,0,0.1); transition:all 0.3s; border-top:4px solid {accent_color};">
            <div style="display:flex; align-items:center; margin-bottom:15px;">
                <div style="font-size:24px; margin-right:12px;">{icon}</div>
                <h3 style="margin:0; font-size:1.2em; color:#1e293b; flex-grow:1;">{viz_name}</h3>
                <span style="background:{accent_color}20; color:{accent_color}; padding:4px 12px; border-radius:20px; font-size:0.85em; font-weight:600;">
                    {category}
                </span>
            </div>
            <p style="margin:0 0 20px 0; color:#64748b; font-size:0.95em; line-height:1.5;">
                {description}
            </p>
            <div style="display:flex; gap:10px;">
                <a href="{viz_url}" target="_blank" 
                   style="flex:1; text-align:center; padding:12px; 
                          background:{accent_color}; color:white; border-radius:8px; 
                          text-decoration:none; font-weight:600; font-size:0.95em;
                          transition:all 0.2s;">
                    🚀 Open Visualization
                </a>
                <button onclick="copyToClipboard('{viz_url}')" 
                        style="padding:12px 20px; background:#f1f5f9; color:#64748b; 
                               border:1px solid #e2e8f0; border-radius:8px; cursor:pointer;
                               font-weight:600; font-size:0.95em; transition:all 0.2s;"
                        data-url="{viz_url}">
                    📋 Copy Link
                </button>
            </div>
        </div>
        '''

dashboard_html += f'''
    </div>
    
    <!-- Usage Instructions -->
    <div style="margin-top:40px; background:rgba(255,255,255,0.1); border-radius:12px; padding:25px; backdrop-filter:blur(10px);">
        <h3 style="margin:0 0 15px 0; color:white;">📖 How to Use This Dashboard</h3>
        <div style="display:grid; grid-template-columns:repeat(auto-fit, minmax(250px, 1fr)); gap:20px;">
            <div style="background:rgba(255,255,255,0.1); padding:20px; border-radius:8px;">
                <div style="font-size:24px; margin-bottom:10px;">🔍</div>
                <h4 style="margin:0 0 10px 0; color:white;">Explore Visualizations</h4>
                <p style="margin:0; color:rgba(255,255,255,0.8); font-size:0.9em;">
                    Click any "Open Visualization" button to explore different aspects of the neural network.
                </p>
            </div>
            <div style="background:rgba(255,255,255,0.1); padding:20px; border-radius:8px;">
                <div style="font-size:24px; margin-bottom:10px;">🎨</div>
                <h4 style="margin:0 0 10px 0; color:white;">Interactive Features</h4>
                <p style="margin:0; color:rgba(255,255,255,0.8); font-size:0.9em;">
                    Use Graphistry's tools to zoom, pan, filter, and inspect individual nodes and edges.
                </p>
            </div>
            <div style="background:rgba(255,255,255,0.1); padding:20px; border-radius:8px;">
                <div style="font-size:24px; margin-bottom:10px;">💾</div>
                <h4 style="margin:0 0 10px 0; color:white;">Share & Save</h4>
                <p style="margin:0; color:rgba(255,255,255,0.8); font-size:0.9em;">
                    Use "Copy Link" to share visualizations or bookmark them for later reference.
                </p>
            </div>
        </div>
    </div>
    
    <div style="margin-top:30px; text-align:center; color:rgba(255,255,255,0.7); font-size:0.9em;">
        <p>Generated with Graphistry • Model: {MODEL_FILE} • {pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
    </div>
</div>
'''

# Add JavaScript for copy functionality
js_code = '''
<script>
function copyToClipboard(url) {
    navigator.clipboard.writeText(url).then(() => {
        const button = event.target;
        const originalText = button.textContent;
        button.textContent = '✓ Copied!';
        button.style.background = '#10b981';
        button.style.color = 'white';
        setTimeout(() => {
            button.textContent = originalText;
            button.style.background = '#f1f5f9';
            button.style.color = '#64748b';
        }, 2000);
    }).catch(err => {
        console.error('Failed to copy: ', err);
    });
}

// Add event listeners to all copy buttons
document.addEventListener('DOMContentLoaded', function() {
    const buttons = document.querySelectorAll('button[data-url]');
    buttons.forEach(button => {
        button.addEventListener('click', function() {
            const url = this.getAttribute('data-url');
            copyToClipboard(url);
        });
    });
});
</script>
'''

# Display the dashboard
from IPython.display import display, HTML
display(HTML(dashboard_html + js_code))

# Save dashboard to file
dashboard_path = '/kaggle/working/complete_dashboard.html'
with open(dashboard_path, 'w') as f:
    final_html = f'''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GGUF Visualization Dashboard - {MODEL_FILE}</title>
    <style>
        body {{
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
            margin: 0;
            padding: 20px;
            background: linear-gradient(135deg, #0f2027 0%, #203a43 50%, #2c5364 100%);
            min-height: 100vh;
        }}
        button:hover {{
            background: #e2e8f0 !important;
        }}
    </style>
</head>
<body>
{dashboard_html}
{js_code}
</body>
</html>'''
    f.write(final_html)

print(f"\n✅ Interactive Dashboard Created!")
print(f"📁 Dashboard saved to: {dashboard_path}")
print(f"\n📋 Dashboard Summary:")
print(f"   • Total Visualizations: {len(all_visualizations)}")
print(f"   • Model: {MODEL_FILE}")
print(f"   • Layers: {n_layers}")
print(f"   • Attention Heads: {attention_heads_count}")
print(f"   • Quantization Blocks: {len(quant_df) if 'quant_df' in locals() else 0}")

# Display download instructions
print(f"\n📥 To download the dashboard:")
print(f"   1. Click on the 'Data' tab in Kaggle")
print(f"   2. Navigate to '/kaggle/working/'")
print(f"   3. Download 'complete_dashboard.html'")

📊 CREATING INTERACTIVE DASHBOARD



✅ Interactive Dashboard Created!
📁 Dashboard saved to: /kaggle/working/complete_dashboard.html

📋 Dashboard Summary:
   • Total Visualizations: 8
   • Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
   • Layers: 28
   • Attention Heads: 896
   • Quantization Blocks: 112

📥 To download the dashboard:
   1. Click on the 'Data' tab in Kaggle
   2. Navigate to '/kaggle/working/'
   3. Download 'complete_dashboard.html'
